In [1]:
# Practice 3-3 (page 24/29)
# https://www.slideshare.net/tadahirotaniguchi0624/3-46861684

In [2]:
import os
import numpy as np

In [3]:
LDCC_DIR = "./text"
groups = [f for f in os.listdir(LDCC_DIR) if os.path.isdir(LDCC_DIR + "/" + f)]

file_names = {}
for group in groups:
    files = os.listdir(LDCC_DIR + "/" + group)
    file_names[group] = files

texts  = []
labels = []

for group in file_names.keys():
    for file_name in file_names[group]:
        with open(os.path.join(LDCC_DIR, group, file_name), 'r') as f:
          lines = f.readlines()
          text = "".join(lines[3:])
        texts.append(text) 
        labels.append(group)
labels = np.array(labels) 

FileNotFoundError: [Errno 2] No such file or directory: './text'

In [4]:
groups

NameError: name 'groups' is not defined

In [5]:
import spacy
nlp = spacy.load('ja_ginza')
docs = list(nlp.pipe(texts, disable=['ner']))

/usr/local/lib/python3.9/site-packages/spacy/util.py:873: UserWarning: [W094] Model 'ja_ginza' (2.1.0) specifies an under-constrained spaCy version requirement: >=2.1.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.1,<3.5.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /usr/local/lib/python3.9/site-packages/ja_ginza/ja_ginza-2.1.0/config.cfg

In [6]:
import spacy

In [11]:
import pkg_resources 
import imp

In [12]:
imp.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.9/site-packages/pkg_resources/__init__.py'>

In [16]:
import glob
import os
import pathlib
import tarfile
import neologdn
import pandas as pd

# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

# rawデータ解凍
tar = tarfile.open("./ldcc-20140209.tar.gz", "r:gz")
tar.extractall(".")
tar.close()

# Dataframeの作成
df = pd.DataFrame(columns=["text", "category"])

for file_path in pathlib.Path("./text").glob("**/*.txt"):
    f_path = pathlib.Path(file_path)
    file_name = f_path.name
    category_name = f_path.parent.name

    if category_name in ['movie-enter', 'sports-watch']:

        # 特殊ファイルはスキップ
        if file_name in ["CHANGES.txt", "README.txt", "LICENSE.txt"]:
            continue

        with open(file_path, "r") as f:
            text_all = f.read()
            text_lines = text_all.split("\n")
            url, time, title, *text = text_lines

            # text前処理
            text = "".join(text)
            text = text.strip()
            text = neologdn.normalize(text)
            text = text.replace(' ', '')
            text = text.replace('　', '')

            df.loc[file_name] = [text,category_name]

# インデックスに使用していたファイル名を削除
df.reset_index(inplace=True,drop=True)

--2022-08-30 09:36:01--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.22MB/s 時間 3.8s       

2022-08-30 09:36:05 (2.22 MB/s) - `ldcc-20140209.tar.gz' へ保存完了 [8855190/8855190]



In [17]:
import scattertext as st
import spacy
from collections import Counter
from itertools import chain
from IPython.display import HTML

# 品詞を絞りこみつつ、unigramの出現回数を集計
class UnigramSelectedPos(st.FeatsFromSpacyDoc):
    """
    品詞の絞り込みを行い、unigramをカウント
    デフォルトの絞り込み品詞は[固有名詞、名詞、動詞、形容詞、副詞]
    """    
    def __init__(self,use_pos=['PROPN', 'NOUN', 'VERB', 'ADJ', 'ADV']):
        super().__init__()
        self._use_pos = use_pos

    def get_feats(self, doc):
        return Counter([c.lemma_ for c in doc if c.pos_ in self._use_pos])

# Corpusの作成
corpus = (st.CorpusFromPandas(df, 
                              category_col='category', 
                              text_col='text',
                              nlp = spacy.load("ja_ginza"),
                              feats_from_spacy_doc=UnigramSelectedPos()
                              )
          .build())

/usr/local/lib/python3.9/site-packages/spacy/util.py:873: UserWarning: [W094] Model 'ja_ginza' (2.1.0) specifies an under-constrained spaCy version requirement: >=2.1.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.1,<3.5.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /usr/local/lib/python3.9/site-packages/ja_ginza/ja_ginza-2.1.0/config.cfg

In [19]:
# spaCyをインポート
import spacy

# nlpオブジェクトを作成
nlp = spacy.blank("en")

# テキストを処理
doc = nlp("This is a sentence.")

# docのテキストをプリント
print(doc.text)

NotImplementedError: [E957] Writing directly to `Language.factories` isn't needed anymore in spaCy v3. Instead, you can use the `@Language.factory` decorator to register your custom component factory or `@Language.component` to register a simple stateless function component that just takes a Doc and returns it.